## Introduction

**Objective**

In this notebook we will build a model that can transform a human portrait photo into an art image. This model will be deployed online as a prototype Tensorflow.js web app.

**Background**

The current method to create a neural style transfer art image involves - choosing a style image, choosing a content image and then running back propogation on this content image for a specified number of iterations. Because this process takes a lot of time it isn't suitable for web deployment. Online users want fast results. 

To ensure fast conversion of photos to art I started by using a neural style transfer alogrithm to create a dataset of 10,200 art images. Here we will train a U-Net cnn using portrait photos as the input data and those art images as the target data.

The art images are part of the Art by Ai dataset. The portrait photos are from the AISegment dataset.

The process used to create the Art by Ai dataset is explained in this notebook:<br>
https://www.kaggle.com/vbookshelf/art-by-ai-how-to-create-the-dataset

**Approach**

- Use U-Net because it was created to ouput images and it's been designed to run fast.
- Use data augmentation (horizontal flipping) to double the size of the available training data.
- Use data generators to ensure that the 13GB kernel RAM limit is not exceeded.

**Results**

The model is able to transform photos to art and the app runs fast. However, the model produces art images that don't match the multi-coloured style of the existing art training images. Duplicating this colouful effect might require more training data and more training epochs.

> Web App:<br>
> http://art.test.woza.work/
> 
> Github:<br>
> https://github.com/vbookshelf/Art-by-Ai-Selfie-Painter

The javascript, html and css code for the app is available on github. For best results, please access the app using the Chrome browser.


<hr>

In [ ]:
# set seeds to ensure repeatability of results
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np
import os
import cv2
#from scipy.misc import imsave

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.io import imread, imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split


# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.listdir('../input')

In [ ]:
# Check for non image files in the folder. These can cause errors later.

# get a list of art images that are in the folder
art_images_list = os.listdir('../input/art-by-ai-neural-style-transfer/content_images/content_images')

for fname in art_images_list:
    extension = fname.split('.')[1]
    
    # if not a jpg image then print the file name
    if extension != 'jpg':
        print(fname)

In [ ]:
IMG_HEIGHT = 400
IMG_WIDTH = 400
IMG_CHANNELS = 3

# Set the number of images to use from the Art by Ai dataset.
SAMPLE_SIZE = 10180

# This batch size will be used for the train and val generators.
# The train gen will output 2*BATCH_SIZE because it augments the images.
# The test gen batch size is set at 1.
BATCH_SIZE = 5

LEARNING_RATE = 0.001

# Tensorflow.js does not support unint8. Therefore we will use int32.
IMG_DTYPE = 'int32'

### Display sample images

In [ ]:
# set up the canvas for the subplots
plt.figure(figsize=(10,10))
plt.axis('Off')

# plt.subplot(nrows, ncols, plot_number)

image_id = '1803261926-00000039.jpg'
folder_id = 1803261926

# == row 1 ==

# image
plt.subplot(1,2,1)
path = '../input/aisegmentcom-matting-human-datasets/matting_human_half/clip_img/' + \
str(folder_id) + '/clip_00000000/' + image_id

image = plt.imread(path)
plt.imshow(image)
plt.title('Source Image', fontsize=12)
plt.axis('off')

# image
plt.subplot(1,2,2)
path = '../input/art-by-ai-neural-style-transfer/content_images/content_images/' + image_id
image = plt.imread(path)
plt.imshow(image)
plt.title('Style Transfer Algo Image', fontsize=12)
plt.axis('off')

plt.show()

In [ ]:
# set up the canvas for the subplots
plt.figure(figsize=(10,10))
plt.axis('Off')

# plt.subplot(nrows, ncols, plot_number)

image_id = '1803281444-00000390.jpg'
folder_id = 1803281444


# == row 1 ==

# image
plt.subplot(1,2,1)
path = '../input/aisegmentcom-matting-human-datasets/matting_human_half/clip_img/' + \
str(folder_id) + '/clip_00000000/' + image_id

image = plt.imread(path)
plt.imshow(image)
plt.title('Source Image', fontsize=12)
plt.axis('off')

# image
plt.subplot(1,2,2)
path = '../input/art-by-ai-neural-style-transfer/content_images/content_images/' + image_id
image = plt.imread(path)
plt.imshow(image)
plt.title('Style Transfer Algo Image', fontsize=12)
plt.axis('off')

plt.show()

<hr>

## Create Dataframes

In [ ]:
# Make a list of 10 test set portraits

test_id_list = ['1803232244-00000007.jpg',
 '1803261926-00000039.jpg',
 '1803281444-00000390.jpg',
 '1803261926-00000434.jpg',
 '1803250811-00000315.jpg',
 '1803261926-00000106.jpg',
 '1803281444-00000166.jpg',
 '1803250811-00000358.jpg',
 '1803250811-00000339.jpg',
 '1803250936-00000547.jpg']


# Load the list of art images in the Art by Ai dataset
art_images_list = os.listdir('../input/art-by-ai-neural-style-transfer/content_images/content_images')

# create a data frame with the file names of all art images
df_art = pd.DataFrame(art_images_list, columns=['image_id'])

# create a test dataframe
df_test = pd.DataFrame(test_id_list, columns=['image_id'])

# Reset the index.
df_test = df_test.reset_index(drop=True)


# Select only rows that are not part of the test set.
# Note the use of ~ to execute 'not in'.
df_data = df_art[~df_art['image_id'].isin(test_id_list)]


print(df_data.shape)
print(df_test.shape)

In [ ]:
df_data.head()

In [ ]:
df_test.head()

## Train Test Split¶

In [ ]:
# train_test_split

# Reduce the number of rows of df_data to speed up training.
# Choose a random sample of rows.
df_data = df_data.sample(SAMPLE_SIZE, random_state=101)

df_train, df_val = train_test_split(df_data, test_size=0.10, random_state=101)


# reset the index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

### Save the dataframes as compressed csv files

Having csv files will allow us to use Pandas chunking to feed images into the generators. Although not essential here, compression is very helpful when working with huge datasets because there's only 4.9GB of disk space available in the Kaggle kernel.

In [ ]:
# save the dataframes as a compressed csv files

df_train.to_csv('df_train.csv.gz', compression='gzip', index=False)
df_val.to_csv('df_val.csv.gz', compression='gzip', index=False)
df_test.to_csv('df_test.csv.gz', compression='gzip', index=False)

In [ ]:
# check if the files were saved
!ls

## Build the Data Generators

The ouput from a generator does not accumulate in memory. Each output batch overwrites the last one. This means that we can feed large amounts of data into a model without running out of RAM and crashing the kernel. There's a 13GB RAM limit when using a GPU.

We will use Pandas chunking and the compressed csv files to feed data into the generators. Using chunking simplifies the code. For example, the last batch that is fed into a generator will be smaller than the others. Pandas chunking will handle this change in batch size automatically which means that we won't need to write code to handle this condition.

Chunking is very useful when the csv file data is too large to be loaded into memory i.e. into a single Pandas dataframe.

**Notes**

> - X_train images are sourced from the AISegment dataset of human portrait photos. The photos are 800x600 therefore, we will need to resize to 400x400 to suit the square shape that U-Net expects.
> - Y_train is the art images associated with the above human portrait photos. These are part of the Art by Ai dataset. The size is 400x300. We will resize to 400x400.
> - Both the portrait photos and their corresponding art images have the same file name.
- The train gen will output a stacked matrix made up of orginal images and those same images that have been flipped horizontally. It outputs a X_train and Y_train matrices that are twice the size of the input batch size. Flipping the images doubles the amount of training data.
- We won't normalize the data.

### [ 1 ] Train Generator

In [ ]:
def train_generator(batch_size=5):
    
    while True:
        
        # load the data in chunks (batches) from  ** df_train.csv.gz **
        for df in pd.read_csv('df_train.csv.gz', chunksize=batch_size):
            
            # get the list of portrait images
            image_id_list = list(df['image_id'])
            # get list of art images
            art_id_list = list(df['image_id'])
            
            
            # X_train
            # =========
            
            # create an empty matrix
            X_orig = np.zeros((len(df), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=IMG_DTYPE)
            X_hflip = np.zeros((len(df), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=IMG_DTYPE)


            for i, image_id in enumerate(image_id_list):

                # select the folder_id from the list
                folder_id = image_id.split('-')[0]

                # set the path to the image
                path = '../input/aisegmentcom-matting-human-datasets/matting_human_half/clip_img/' + \
                str(folder_id) + '/clip_00000000/' + image_id

                # read the file as an array
                image = plt.imread(path)
                # resize the image
                image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
                
                # original image
                X_orig[i] = image
                # flip image horizontally
                X_hflip[i] = np.fliplr(image)

                # stack the matrices to form X_train
                X_train = np.vstack((X_orig, X_hflip))

            
            
            # Y_train
            # =========
            
            
            # create an empty matrix
            Y_orig = np.zeros((len(df), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=IMG_DTYPE)
            Y_hflip = np.zeros((len(df), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=IMG_DTYPE)


            for i, image_id in enumerate(art_id_list):

                # set the path to the image
                path = '../input/art-by-ai-neural-style-transfer/content_images/content_images/' + image_id

                # read the file as an array
                #image = imread(path)
                # read the image using skimage
                image = imread(path)
                # resize the image
                image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
                
                # original image
                Y_orig[i] = image
                # flip image horizontally
                Y_hflip[i] = np.fliplr(image)

                # stack the matrices to form Y_train
                Y_train = np.vstack((Y_orig, Y_hflip))
        
        
            yield X_train, Y_train

### Sanity check the train generator

In [ ]:
# Check the generator

# initialize
train_gen = train_generator(batch_size=5)

In [ ]:
# run the generator
X_train, Y_train = next(train_gen)

print(X_train.shape)
print(Y_train.shape)

In [ ]:
# print an image image from X_train

# can also write
#img = X_train[0]

img = X_train[0,:,:,:]
plt.imshow(img)

plt.show()

In [ ]:
# print an image from Y_train

# can also write
#img = Y_train[0]

img = Y_train[0,:,:,:]
plt.imshow(img)

plt.show()

### [ 2 ] Val Generator

In [ ]:
def val_generator(batch_size=5):
    
    while True:
        
        # load the data in chunks (batches) from  ** df_val.csv.gz **
        for df in pd.read_csv('df_val.csv.gz', chunksize=batch_size):
            
            # get the list of portrait images
            image_id_list = list(df['image_id'])
            # get list of art images
            art_id_list = list(df['image_id'])
            
            
            # X_val
            # =========
            
            # create an empty matrix
            X_val = np.zeros((len(df), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=IMG_DTYPE)


            for i, image_id in enumerate(image_id_list):

                # select the folder_id from the list
                folder_id = image_id.split('-')[0]

                # set the path to the image
                path = '../input/aisegmentcom-matting-human-datasets/matting_human_half/clip_img/' + \
                str(folder_id) + '/clip_00000000/' + image_id

                # read the file as an array
                image = plt.imread(path)
                # resize the image
                image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)


                # insert the image into X_val
                X_val[i] = image
                
            
            
            # Y_val
            # =========
            
            
            # create an empty matrix
            Y_val = np.zeros((len(df), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=IMG_DTYPE)


            for i, image_id in enumerate(art_id_list):

                # set the path to the image
                path = '../input/art-by-ai-neural-style-transfer/content_images/content_images/' + image_id

                # read the file as an array
                #image = imread(path)
                # read the image using skimage
                image = imread(path)
                # resize the image
                image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

                # insert the image into X_val
                Y_val[i] = image
        
        
            yield X_val, Y_val

### Sanity check the val generator

In [ ]:
# Check the generator

# initialize
val_gen = val_generator(batch_size=5)

In [ ]:
# run the generator
X_val, Y_val = next(val_gen)

print(X_val.shape)
print(Y_val.shape)

In [ ]:
# print an image from X_val

# can also write
#img = X_val[0]

img = X_val[0,:,:,:]
plt.imshow(img)

plt.show()

In [ ]:
# print an image from Y_val

# can also write
#img = Y_val[0]

img = Y_val[0,:,:,:]
plt.imshow(img)

plt.show()

### [ 3 ] Test Generator

In [ ]:
def test_generator(batch_size=1):
    
    while True:
        
        # load the data in chunks (batches) from  ** df_test.csv.gz **
        for df in pd.read_csv('df_test.csv.gz', chunksize=batch_size):
            
            # get the list of portrait images
            image_id_list = list(df['image_id'])
            
            
            # X_test
            # =========
            
            # create an empty matrix
            X_test = np.zeros((len(df), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=IMG_DTYPE)


            for i, image_id in enumerate(image_id_list):

                # select the folder_id from the list
                folder_id = image_id.split('-')[0]

                # set the path to the image
                path = '../input/aisegmentcom-matting-human-datasets/matting_human_half/clip_img/' + \
                str(folder_id) + '/clip_00000000/' + image_id

                # read the file as an array
                image = imread(path)
                # resize the image
                image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)


                # insert the image into X_test
                X_test[i] = image
                
            
            yield X_test

### Sanity check the test generator

In [ ]:
# Test the generator

# initialize
test_gen = test_generator(batch_size=1)

In [ ]:
# run the generator
X_test = next(test_gen)

print(X_test.shape)

In [ ]:
# print the first image in X_test

# can also write
#img = X_test[0]

img = X_test[0,:,:,:]
plt.imshow(img)

plt.show()

## U-Net Model Architecture

> U-Net: Convolutional Networks for Biomedical Image Segmentation<br>
> Olaf Ronneberger, Philipp Fischer, Thomas Brox<br>
> https://arxiv.org/abs/1505.04597

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input, UpSampling2D
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adam

import tensorflow as tf


In [ ]:
# source: https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277
# Modified to ouput an image with 3 channels.


inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)

outputs = Conv2D(3, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

model = Model(inputs=[inputs], outputs=[outputs])


model.summary()

## Train the Model

In [ ]:
num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = BATCH_SIZE
val_batch_size = BATCH_SIZE

# Test batch size will be 1.

# determine num train steps
train_steps = np.ceil(num_train_samples / train_batch_size)
# determine num val steps
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
# Initialize the generators
train_gen = train_generator(batch_size=BATCH_SIZE)
val_gen = val_generator(batch_size=BATCH_SIZE)


model.compile(Adam(lr=LEARNING_RATE), loss='mean_squared_error')


filepath = "model.h5"

earlystopper = EarlyStopping(patience=3, verbose=1)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min')

callbacks_list = [earlystopper, checkpoint]

history = model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=10, 
                              validation_data=val_gen, validation_steps=val_steps,
                             verbose=1,
                             callbacks=callbacks_list)

## Plot the Training Curves

In [ ]:
import matplotlib.pyplot as plt

mean_squared_error = history.history['loss']
val_mean_squared_error = history.history['val_loss']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(mean_squared_error) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, mean_squared_error, 'bo', label='Training mse')
plt.plot(epochs, val_mean_squared_error, 'b', label='Validation mse')
plt.title('Training and validation mse')
plt.legend()
plt.figure()

## Make a Prediction

In [ ]:

# initialize the test generator
test_gen = test_generator(batch_size=1)

# use the best epoch
model.load_weights(filepath = 'model.h5')

preds = model.predict_generator(test_gen, 
                                steps=len(df_test), 
                                verbose=1)

# check the max and min predicted pixel values
print('\n')
print(preds.max())
print(preds.min())

In [ ]:
# Clip the predicted pixel values to between 0 and 255.

preds = np.clip(preds, 0, 255).astype(IMG_DTYPE)


# check the max and min predicted pixel values again
print(preds.max())
print(preds.min())

## Display Test Set Results

In [ ]:
# set up the canvas for the subplots
plt.figure(figsize=(20,20))
plt.axis('Off')

# Our subplot will contain 3 rows and 3 columns
# plt.subplot(nrows, ncols, plot_number)

image_id = '1803261926-00000039.jpg'
folder_id = 1803261926

# == row 1 ==

# image
plt.subplot(1,3,1)
path = '../input/aisegmentcom-matting-human-datasets/matting_human_half/clip_img/' + \
str(folder_id) + '/clip_00000000/' + image_id

image = plt.imread(path)
plt.imshow(image)
plt.title('Source Image', fontsize=20)
plt.axis('off')

# image
plt.subplot(1,3,2)
path = '../input/art-by-ai-neural-style-transfer/content_images/content_images/' + image_id
image = plt.imread(path)
plt.imshow(image)
plt.title('Style Transfer Algo Image', fontsize=20)
plt.axis('off')


# image
plt.subplot(1,3,3)
image = preds[1]
plt.imshow(image)
plt.title('CNN Predicted Image', fontsize=20)
plt.axis('off')

plt.show()#

In [ ]:
# set up the canvas for the subplots
plt.figure(figsize=(20,20))
plt.axis('Off')

# Our subplot will contain 3 rows and 3 columns
# plt.subplot(nrows, ncols, plot_number)

image_id = '1803281444-00000390.jpg'
folder_id = 1803281444

# == row 1 ==

# image
plt.subplot(1,3,1)
path = '../input/aisegmentcom-matting-human-datasets/matting_human_half/clip_img/' + \
str(folder_id) + '/clip_00000000/' + image_id

image = plt.imread(path)
plt.imshow(image)
plt.title('Source Image', fontsize=20)
plt.axis('off')

# image
plt.subplot(1,3,2)
path = '../input/art-by-ai-neural-style-transfer/content_images/content_images/' + image_id
image = plt.imread(path)
plt.imshow(image)
plt.title('Style Transfer Algo Image', fontsize=20)
plt.axis('off')


# image
plt.subplot(1,3,3)
image = preds[2]
plt.imshow(image)
plt.title('CNN Predicted Image', fontsize=20)
plt.axis('off')

plt.show()

The app will re-size the predicted image to 320x240 before displaying it on the web page. 

## Convert the Model to Tensorflow.js

One challenge that I find with Tensorflow.js is that the model conversion process is not robust. Here I've had to use workarounds to address two errors that ocurred during model conversion. This technology is still maturing so these bugs are to be expected. Even with these glitches, Tensorflow.js is still a fantastic tool.

In [ ]:
# Stackoverflow solutions to errors:

# https://stackoverflow.com/questions/49932759/pip-10-and-apt-how-to-avoid-cannot-uninstall
# -x-errors-for-distutils-packages

# https://stackoverflow.com/questions/56003095/no-add-to-collection-was-found-when-
# using-tensorflowjs-converter


In [ ]:
# Install tensorflowjs.
# Don't use the latest version. Instead install version 1.1.2

# --ignore-installed is added to fix an error.

!pip install tensorflowjs==1.1.2 --ignore-installed

In [ ]:
# Use the command line conversion tool to convert the model

!tensorflowjs_converter --input_format keras model.h5 tfjs/model

In [ ]:
# check that the folder containing the tfjs model files has been created
!ls

## Conclusion

Here we used this workflow to generate art. I believe that this workflow can be adapted for use in other areas like medicine or forensics - in fact any use-case that requires a web based Ai tool that outputs an image.

Thank you for reading.